How do I train the kaggle data in colab?:

my reference

https://www.kaggle.com/qgh1223/inceptionv3-siamesenet


1.Upload the key of kaggle into the kagggle

In [0]:
import os

# Upload the API token.
def get_kaggle_credentials():
    token_dir = os.path.join(os.path.expanduser("~"),".kaggle")
    token_file = os.path.join(token_dir, "kaggle.json")
    if not os.path.isdir(token_dir):
        os.mkdir(token_dir)
    try:
        with open(token_file,'r') as f:
            pass
    except IOError as no_file:
        try:
            from google.colab import files
        except ImportError:
            raise no_file

        uploaded = files.upload()

        if "kaggle.json" not in uploaded:
            raise ValueError("You need an API key! see: "
                           "https://github.com/Kaggle/kaggle-api#api-credentials")
        with open(token_file, "wb") as f:
            f.write(uploaded["kaggle.json"])
        os.chmod(token_file, 600)

get_kaggle_credentials()

In [ ]:
2.Download the train data and test data

In [0]:
!pip install kaggle
import kaggle
# This will download the data (600 MB)
competition_name = 'Humpback-Whale-Identification'

kaggle.api.competition_download_file(competition_name, file_name='train.zip')
kaggle.api.competition_download_file(competition_name, file_name='test.zip')
kaggle.api.competition_download_file(competition_name, file_name='train.csv')

train.zip: Skipping, found more recently modified local copy (use --force to force download)


  0%|          | 5.00M/1.35G [00:00<00:51, 28.1MB/s]

100%|██████████| 1.35G/1.35G [00:09<00:00, 150MB/s]


100%|██████████| 594k/594k [00:00<00:00, 83.1MB/s]

3.Move the data to input

In [0]:
!mkdir -p input/train/&& mkdir -p input/test
!mv train.zip input/train/train.zip&& mv test.zip input/test/test.zip
!mv train.csv input/train.csv

4.unzip the data

In [0]:
!cd input/train/ && unzip train.zip
!cd input/test/ && unzip test.zip

Archive:  train.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of train.zip or
        train.zip.zip, and cannot find train.zip.ZIP, period.
Archive:  test.zip
replace 8025e1774.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: 8025e1774.jpg           
  inflating: 18314d823.jpg           
  inflating: ecad2e839.jpg           
  inflating: 262f5cb0c.jpg           
  inflating: 0f89e1816.jpg           
  inflating: 0fe281ec5.jpg           
  inflating: 4e09edc85.jpg           
  inflating: 3180e1cd5.jpg           
  inflating: 8900e3f78.jpg           
  inflating: 92835da1f.jpg           
  inflating: 5d0ca9ac4.jpg           
  inflating: b73949ea6.jpg           
  inflating: c3957db12.jpg           
  inflating: bd0479d5b.jpg

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import os
import matplotlib.pyplot as plt
import random
from keras import backend as K

Using TensorFlow backend.


In [0]:
from keras.models import Model,Sequential
from keras.layers import Activation, Add, Input, Flatten, Dense, Dropout, Lambda,Conv2D, BatchNormalization,\
                         GlobalMaxPooling2D,MaxPooling2D,Concatenate,Reshape
from keras.optimizers import Adam,RMSprop
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.utils import Sequence
from keras import regularizers
import PIL
from PIL import Image
from imgaug import augmenters as iaa
import imgaug as ia

5.Define the function for crop the image

In [0]:
def center_crop(x, center_crop_size):
    centerw, centerh = x.shape[0] // 2, x.shape[1] // 2
    halfw, halfh = center_crop_size[0] // 2, center_crop_size[1] // 2
    cropped = x[centerw - halfw : centerw + halfw,
                 centerh - halfh : centerh + halfh, :]

    return cropped

def scale_byRatio(img_path, ratio=1.0, return_width=299, crop_method=center_crop):
    # Given an image path, return a scaled array
    img = cv2.imread(img_path)
    h = img.shape[0]
    w = img.shape[1]
    shorter = min(w, h)
    longer = max(w, h)
    img_cropped = crop_method(img, (shorter, shorter))
    img_resized = cv2.resize(img_cropped, (return_width, return_width), interpolation=cv2.INTER_CUBIC)
    img_rgb = img_resized
    img_rgb[:, :, [0, 1, 2]] = img_resized[:, :, [2, 1, 0]]

    return img_rgb

In [0]:
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

# Define our sequence of augmentation steps that will be applied to every image
# All augmenters with per_channel=0.5 will sample one value _per image_
# in 50% of all cases. In all other cases they will sample new values
# _per channel_.
seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        # crop images by -5% to 10% of their height/width
        sometimes(iaa.CropAndPad(
            percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        )),
        sometimes(iaa.Affine(
            scale={"x": (0.85, 1.15), "y": (0.85, 1.5)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.15, 0.15), "y": (-0.15, 0.15)}, # translate by -20 to +20 percent (per axis)
            rotate=(-15, 15), # rotate by -45 to +45 degrees
            shear=(-5, 5), # shear by -16 to +16 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
        # execute 0 to 5 of the following (less important) augmenters per image
        # don't execute all of them, as that would often be way too strong
        iaa.SomeOf((0, 3),
            [
                iaa.OneOf([
                    iaa.GaussianBlur((0, 2.0)), # blur images with a sigma between 0 and 3.0
                    iaa.AverageBlur(k=(1, 5)), # blur image using local means with kernel sizes between 2 and 7
                    iaa.MedianBlur(k=(1, 5)), # blur image using local medians with kernel sizes between 2 and 7
                ]),

                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.03*255), per_channel=0.5), # add gaussian noise to images
                iaa.OneOf([
                    iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                    iaa.CoarseDropout((0.03, 0.15), size_percent=(0.01, 0.03), per_channel=0.2),
                ]),
                iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)

                iaa.ContrastNormalization((0.3, 1.0), per_channel=0.5), # improve or worsen the contrast
                iaa.Grayscale(alpha=(0.0, 1.0)),
                sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
            ],
            random_order=True
        )
    ],
    random_order=True
)

6. Define the function for get the train data list and the test data list

In [0]:
def kind_list(imgdata):
    kindlist=imgdata.groupby('Id').size()
    return kindlist.index

def fetch_all_kind_list(imgdata):
    kindlist=kind_list(imgdata)
    kindimgpathlist=[]
    for kind in kindlist:
        kindimgpathlist.append(list(imgdata['Image'][imgdata['Id']==kind]))
    return kindimgpathlist,kindlist

def fetch_kind_list_split(kindimgpathlist,split_size=0.8):
    trainkindimgpathlist=[]
    validkindimgpathlist=[]
    for pathlist in kindimgpathlist:
        if(len(pathlist)<=3):
            trainkindimgpathlist.append(pathlist)
            validkindimgpathlist.append(pathlist)
        else:
            trainkindimgpathlist.append(pathlist[:int(len(pathlist)*split_size)])
            validkindimgpathlist.append(pathlist[int(len(pathlist)*split_size):])
    return trainkindimgpathlist,validkindimgpathlist

7.Define the  train data generator from the path

In [0]:
def siamese_img_gen(BASE_DIR,img_width,img_height,imgpathlist,
                    contrast_times=2,batch_size=50,scale_ratio=1.0,crop_method=center_crop):
    while(True):
        imglist1=[]
        imglist2=[]
        labellist=[]
        for i in range(batch_size):
          for j in range(contrast_times):
                rndid=random.randint(0,len(imgpathlist)-1)
                if(i%2==0):
                    #print(len(kindimgpathlist[rndid]))
                    pair=np.random.randint(0,len(imgpathlist[rndid]),2)
                    imgpath1=imgpathlist[rndid][pair[0]]
                    imgpath2=imgpathlist[rndid][pair[1]]
                    labellist.append(1)
                else:
                    rndid1=random.randint(0,len(imgpathlist[rndid])-1)
                    imgpath1=imgpathlist[rndid][rndid1]
                    index1=random.choice([num for num in range(len(imgpathlist)) if num not in [rndid]])
                    rndid2=random.randint(0,len(imgpathlist[index1])-1)
                    imgpath2=imgpathlist[index1][rndid2]
                    labellist.append(0)
                img1=scale_byRatio(BASE_DIR+imgpath1, ratio=scale_ratio, return_width=img_width,
                                crop_method=crop_method)
                img2=scale_byRatio(BASE_DIR+imgpath2, ratio=scale_ratio, return_width=img_width,
                                crop_method=crop_method)
                imglist1.append(img1)
                imglist2.append(img2)
        imglist1 = np.array(imglist1,dtype=np.uint8)
        imglist1 = seq.augment_images(imglist1)
        imglist2 = np.array(imglist2,dtype=np.uint8)
        imglist2 = seq.augment_images(imglist2)
        yield ([np.asarray(imglist1),np.asarray(imglist2)],np.asarray(labellist))

In [0]:
IMG_WIDTH=IMG_HEIGHT=384
BASE_DIR='./input/train/'
labelpath='./input/train.csv'
df=pd.read_csv(labelpath)
img_shape=(IMG_WIDTH,IMG_HEIGHT,3)

In [0]:
LEARNING_RATE = 0.001
BATCH_SIZE = 32
FINE_TUNE = False

In [0]:
def subblock(x, filter, **kwargs):
    x = BatchNormalization()(x)
    y = x
    y = Conv2D(filter, (1, 1), activation='relu', **kwargs)(y)  # Reduce the number of features to 'filter'
    y = BatchNormalization()(y)
    y = Conv2D(filter, (3, 3), activation='relu', **kwargs)(y)  # Extend the feature field
    y = BatchNormalization()(y)
    y = Conv2D(K.int_shape(x)[-1], (1, 1), **kwargs)(y)  # no activation # Restore the number of original features
    y = Add()([x, y])  # Add the bypass connection
    y = Activation('relu')(y)
    return y

8.Build the siamese network mode 

In [13]:
def build_model(lr, l2, activation='sigmoid'):  
    regul = regularizers.l2(l2)
    optim = Adam(lr=lr)
    kwargs = {'padding': 'same', 'kernel_regularizer': regul}

    input1 = Input(shape=img_shape)  # 384x384x3
    x = Conv2D(64, (7, 7), strides=2, activation='relu', **kwargs)(input1)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 96x96x64
    for _ in range(2):
        x = BatchNormalization()(x)
        x = Conv2D(64, (3, 3), activation='relu', **kwargs)(x)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 48x48x64
    x = BatchNormalization()(x)
    x = Conv2D(128, (1, 1), activation='relu', **kwargs)(x)  # 48x48x128
    for _ in range(4):
        x = subblock(x, 64, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 24x24x128
    x = BatchNormalization()(x)
    x = Conv2D(256, (1, 1), activation='relu', **kwargs)(x)  # 24x24x256
    for _ in range(4):
        x = subblock(x, 64, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 12x12x256
    x = BatchNormalization()(x)
    x = Conv2D(384, (1, 1), activation='relu', **kwargs)(x)  # 12x12x384
    for _ in range(4):
        x = subblock(x, 96, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 6x6x384
    x = BatchNormalization()(x)
    x = Conv2D(512, (1, 1), activation='relu', **kwargs)(x)  # 6x6x512
    for _ in range(4):
        x = subblock(x, 128, **kwargs)

    x = GlobalMaxPooling2D()(x)  # 512
    branch_model = Model(input1, x)

    
    mid = 32
    xa_inp = Input(shape=branch_model.output_shape[1:])
    xb_inp = Input(shape=branch_model.output_shape[1:])
    x1 = Lambda(lambda x: x[0] * x[1])([xa_inp, xb_inp])
    x2 = Lambda(lambda x: x[0] + x[1])([xa_inp, xb_inp])
    x3 = Lambda(lambda x: K.abs(x[0] - x[1]))([xa_inp, xb_inp])
    x4 = Lambda(lambda x: K.square(x))(x3)
    x = Concatenate()([x1, x2, x3, x4])
    x = Reshape((4, branch_model.output_shape[1], 1), name='reshape1')(x)

    # Per feature NN with shared weight is implemented using CONV2D with appropriate stride.
    x = Conv2D(mid, (4, 1), activation='relu', padding='valid')(x)
    x = Reshape((branch_model.output_shape[1], mid, 1))(x)
    x = Conv2D(1, (1, mid), activation='linear', padding='valid')(x)
    x = Flatten(name='flatten')(x)

    # Weighted sum implemented as a Dense layer.
    x = Dense(1, use_bias=True, activation=activation, name='weighted-average')(x)
    distance_model = Model([xa_inp, xb_inp], x, name='distance')

    
    img_a = Input(shape=img_shape)
    img_b = Input(shape=img_shape)
    
    ba = branch_model(img_a)
    bb = branch_model(img_b)
    
    x = distance_model([ba, bb])
    
    model = Model([img_a, img_b], x)

    model.compile(optim, loss='binary_crossentropy', metrics=['acc'])
    
    return model, branch_model, distance_model

model, branch_model, distance_model = build_model(6e-5, 0)
branch_model.summary()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 384, 384, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 192, 192, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 96, 96, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 96, 64)   256         max_pooling2d_1[0][0]            
_____________________________________

In [0]:
kindimgpathlist,kindlist=fetch_all_kind_list(df)
trainkindimgpathlist,validkindimgpathlist=fetch_kind_list_split(kindimgpathlist)

9.Begin train the data

In [0]:
model.load_weights('siamese.h5')
callbacks=[
    EarlyStopping(monitor='val_loss', patience=15, verbose=1),
    ReduceLROnPlateau(monitor='val_loss',patience=5,min_lr=1e-9,verbose=1,mode='min'),
    ModelCheckpoint('siamese.h5',monitor='val_loss',save_best_only=True,verbose=1)
]

history=model.fit_generator(siamese_img_gen(BASE_DIR,IMG_WIDTH,IMG_HEIGHT,
                                            trainkindimgpathlist,batch_size=20),
                            steps_per_epoch=5,
                            epochs=30,
                            verbose = 1,
                            validation_data=siamese_img_gen(BASE_DIR,IMG_WIDTH,IMG_HEIGHT,
                                                            validkindimgpathlist,contrast_times=10,batch_size=5),
                            validation_steps=10,
                            callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/30
5/5 [==============================] - 628s 126s/step - loss: 0.6898 - acc: 0.5350 - val_loss: 0.6801 - val_acc: 0.5820

Epoch 00001: val_loss improved from inf to 0.68006, saving model to siamese.h5
Epoch 2/30
5/5 [==============================] - 529s 106s/step - loss: 0.6948 - acc: 0.5200 - val_loss: 0.6933 - val_acc: 0.5720

Epoch 00002: val_loss did not improve from 0.68006
Epoch 3/30
5/5 [==============================] - 519s 104s/step - loss: 0.7009 - acc: 0.4750 - val_loss: 0.6837 - val_acc: 0.6120

Epoch 00003: val_loss did not improve from 0.68006
Epoch 4/30
5/5 [==============================] - 539s 108s/step - loss: 0.6977 - acc: 0.5050 - val_loss: 0.7006 - val_acc: 0.5620

Epoch 00004: val_loss did not improve from 0.68006
Epoch 5/30
5/5 [==============================] - 535s 107s/step - loss: 0.6912 - acc: 0.5450 - val_loss: 0.7436 -

10. The follow is predict the test data

In [0]:
model_path = 'siamese.h5'
model.load_weights(model_path)

In [0]:
from glob import glob
from skimage.io import imread

TEST_DIR = './input/test'
batch_size = 1000
classes = df.Id.unique()
test_files = glob(os.path.join('./input/test/','*.jpg'))
max_idx = len(test_files)
test_df = pd.DataFrame({'path': test_files[:max_idx]})
test_df['Image'] = test_df.path.map(lambda x: x.split('/')[3]) 

In [0]:
def test_img_gen(img_test,IMG_ROW,IMG_COL,imgpathlist,batch_size,current_index):
  imglist1=[]
  imglist2=[]
  N = len(imgpathlist)
    
  if N >= (current_index + batch_size):
    current_batch_size = batch_size
  else:
    current_batch_size = N - current_index
  for i in range(current_index,current_index+current_batch_size):
    rndid=random.randint(0,len(imgpathlist[i])-1)
    imgpath=imgpathlist[i][rndid]
    img=scale_byRatio(BASE_DIR+imgpath, ratio=1.0, return_width=IMG_WIDTH,
                                crop_method=center_crop)
    imglist1.append(img_test)
    imglist2.append(img)
  return ([np.asarray(imglist1),np.asarray(imglist2)])



In [ ]:
for i in range(max_idx):
  print("Indexes: %i/%i"%(i, max_idx))
  img_test=scale_byRatio(test_files[i], ratio=1.0, return_width=IMG_WIDTH,
                                crop_method=center_crop)
  labels=[]
  for idx in range(0,len(classes),batch_size):
    predicted_labels = model.predict(test_img_gen(img_test=img_test,
                   IMG_ROW=IMG_WIDTH,IMG_COL=IMG_HEIGHT,imgpathlist=kindimgpathlist,batch_size=batch_size,current_index=idx),verbose=1)
    labels.extend(predicted_labels)
  test_df.loc[i, 'Id']= ' '.join(kindlist[np.argsort(np.reshape(np.array(labels),(len(classes),)))[:5][::-1]])
submission1 = pd.DataFrame()
submission1 = pd.concat([submission1, test_df[["Image", "Id"]]])

submission1.to_csv('submission.csv', index=False, header=True) 